**Importing Useful Libraries**

In [1]:
import cv2
import numpy as np
from numpy.linalg import inv
import math
import sys
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import xlrd
from skimage.measure import structural_similarity as ssim
from scipy.stats.stats import pearsonr

**Loading Raw and Original Images(Bayer Canon Images without Noise are used)**

In [9]:
images=[]
oimages=[]
for i in range(1,85,1):
    images.append(cv2.imread("./DIBR_Data/"+str(i)+".png",0));

In [24]:
geo_distortion=[]
count=[]
for img in images:
    ii=np.empty((len(img),len(img[0])),dtype=np.uint8);
    c=5
    for i in range(0,len(img),1):
        for j in range(0,len(img[0]),1):
            if img[i][j]!=0:
                ii[i][j]=255;
            else:
                ii[i][j]=0;
                c=c+1;
    geo_distortion.append(ii);
    count.append(c);

In [18]:
loc = ("./DMOS_DIBR.xls"); 
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(0) 
dmos=[]
for i in range(0,84,1):
    dmos.append(sheet.cell_value(i, 0)) 

In [25]:
print(len(dmos))
print(len(count))
pearsonr(count,dmos)

84
84


(-0.56442296714646556, 2.2482519793661132e-08)

In [ ]:
#2nd method
geo_distortion=[]
count=[]
for img in images:
    ii=np.zeros((len(img),len(img[0])),dtype=np.float64);
    c=5
    for i in range(0,len(img),1):
        for j in range(0,len(img[0]),1):
            if img[i][j][1]!=0:
                ii[i][j]=255.0;
            else:
                c=c+1;
    geo_distortion.append(ii);
    count.append(c);

In [ ]:
for i in range(1,85,1):
    cv2.imwrite(str(i)+'_'+str(count[i-1])+'.jpg',geo_distortion[i-1])

**Showing Raw Images**

In [ ]:
med_images=[]
for i in images:
    med_images.append(cv2.medianBlur(i,5));

In [ ]:
geo_distortion_med=[]
count_med=[]
for img in med_images:
    ii=np.zeros((len(img),len(img[0])),dtype=np.float64);
    c=5
    for i in range(0,len(img),1):
        for j in range(0,len(img[0]),1):
            if img[i][j][1]!=0:
                ii[i][j]=255.0;
            else:
                c=c+1;
    geo_distortion_med.append(ii);
    count_med.append(c);

In [ ]:
pearsonr(count_med,dmos)

In [43]:
kernel = np.ones((5,5),np.uint8)
closing_images=[]
for i in images:
    closing_images.append(cv2.morphologyEx(i, cv2.MORPH_CLOSE, kernel));

In [45]:
geo_distortion_closing=[]
count_closing=[]
for img in closing_images:
    ii=np.zeros((len(img),len(img[0])),dtype=np.float64);
    c=5
    for i in range(0,len(img),1):
        for j in range(0,len(img[0]),1):
            if img[i][j]!=0:
                ii[i][j]=255.0;
            else:
                c=c+1;
    geo_distortion_closing.append(ii);
    count_closing.append(c);

In [46]:
for i in range(1,85,1):
    cv2.imwrite("Closing_"+str(i)+'_'+str(count_closing[i-1])+'.jpg',geo_distortion_closing[i-1])
pearsonr(count_closing,dmos)

(-0.55993230295768048, 3.0657039253594429e-08)

In [57]:
# Sub closed image with original image
subtracted_images=[]
for k in range(0,84,1):
    temp=np.zeros((len(images[0]),len(images[0][0])))
    for i in range(0,len(images[0]),1):
        for j in range(0,len(images[0][0]),1):
                temp[i,j]=abs(closing_images[k][i,j]-images[k][i,j])-images[k][i,j];
    subtracted_images.append(temp);

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in ubyte_scalars
  import sys


In [58]:
for i in range(1,85,1):
    cv2.imwrite("Subtracted_"+str(i)+'_'+str(count_closing[i-1])+'.jpg',subtracted_images[i-1])

In [55]:
sum_in_sub_dark=[]
for k in range(0,84,1):
    temp=0
    for i in range(0,len(images[0]),1):
        for j in range(0,len(images[0][0]),1):
                temp+=subtracted_images[k][i,j];
    sum_in_sub_dark.append(temp);

In [56]:
pearsonr(sum_in_sub_dark,dmos)

(-0.40356214251047223, 0.00014093659087077598)

In [34]:
sift = cv2.xfeatures2d.SIFT_create()
count_original_sift=[]
for img in images:
    kp, des = sift.detectAndCompute(img,None)
    count_original_sift.append(len(kp));

In [30]:
count_distorted_sift=[]
for img in geo_distortion:
    kp, des = sift.detectAndCompute(img,None)
    count_distorted_sift.append(len(kp));

In [42]:
print(4//8)
count_ratio=[]
for i in range(0,84,1):
    count_ratio.append(float(count_distorted_sift[i])/float(count_original_sift[i]));
print(count_ratio)
pearsonr(count_ratio,dmos)
# print(len(count_distorted_sift)

0
[0.0, 0.0, 0.18411967779056387, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0832282471626734, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09543742151527836, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1654733492442323, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04342017368069472, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.025847599865726753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02904841402337229, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.041708043694141016, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19743935309973046, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11243386243386243, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14500197550375346, 0.0008485362749257531, 0.0, 0.0, 0.0, 0.0, 0.0, 0.31051009928106815, 0.004568106312292359, 0.0, 0.0, 0.0]


(-0.56585934886324529, 2.0339521995648248e-08)

In [ ]:
bilateral_images=[]
for i in closing_images:
    bilateral_images.append(cv2.bilateralFilter(i,9,75,75));

In [ ]:
def mse(o,p):
    error=0.0;
    for k in range(0,o.shape[2],1):
        for i in range(0,o.shape[0],1):
            for j in range(0,o.shape[1],1):
                e=o[i,j,k]-p[i,j,k];
                error+=e*e;
    return error/(o.shape[0]*o.shape[1]*o.shape[2])

def psnr(mse):
	return 10*math.log10((255*255)/mse)

In [ ]:
Psnr=[]
for i in range(0,84,1):
    m=mse(images[i],bilateral_images[i]);
    Psnr.append(psnr(m));

In [ ]:
for i in range(1,85,1):
    cv2.imwrite(str(i)+'_1.jpg',closing_images[i-1][:,:,0])
    cv2.imwrite(str(i)+'_2.jpg',closing_images[i-1][:,:,1])
    cv2.imwrite(str(i)+'_3.jpg',closing_images[i-1][:,:,2])

In [ ]:
print(Psnr)

In [ ]:
for i in range(0,len(images),1):
    for x in range(1,images[i].shape[0],2):
        images[i][x,:,0]=0;
    for x in range(1,images[i].shape[1],2):
        images[i][:,x,0]=0;
 
    for x in range(0,images[i].shape[0],1):
        for y in range(0,images[i].shape[1],1):
            if (x%2==0 and y%2==0):
                images[i][x,y,1]=0;
            elif (x%2==1 and y%2==1):
                images[i][x,y,1]=0;
    
    for x in range(0,images[i].shape[0],2):
        images[i][x,:,2]=0;
    for x in range(0,images[i].shape[1],2):
        images[i][:,x,2]=0;
        

**Defining Demosaicing function. Bilinear Approach has been used for interpolation.**

In [ ]:
def Demosaicing(img):
#For Red Layer
    
    for i in range(0,img.shape[0],2):
        for j in range(1,img.shape[1],2):
            if j+1<img.shape[1]:
                img[i,j,0]=int(math.ceil((img[i,j-1,0]/2.0)+(img[i,j+1,0]/2.0)));
    
    for i in range(1,img.shape[0],2):
        for j in range(0,img.shape[1],2):
            if i+1<img.shape[0]:
                img[i,j,0]=int(math.ceil((img[i-1,j,0]/2.0)+(img[i+1,j,0]/2.0)));
    
    for i in range(1,img.shape[0],2):
        for j in range(1,img.shape[1],2):
            if i+1<img.shape[0] and j+1<img.shape[1]:
                img[i,j,0]=int(math.ceil((img[i-1,j-1,0]/4.0)+(img[i-1,j+1,0]/4.0)+(img[i+1,j-1,0]/4.0)+(img[i+1,j+1,0]/4.0)));
    
    if img.shape[0]%2==0:
        img[img.shape[0]-1,:,0]=img[img.shape[0]-2,:,0]
    
    if img.shape[1]%2==0:
        img[:,img.shape[1]-1,0]=img[:,img.shape[1]-2,0]
    
    if img.shape[0]%2==0 and img.shape[1]%2==0:
        img[img.shape[0]-1,img.shape[1]-1,0]=img[img.shape[0]-2,img.shape[1]-2,0]

#For Green Layer
    
    for i in range(1,img.shape[0],2):
        for j in range(1,img.shape[1],2):
            if i+1<img.shape[0] and j+1<img.shape[1]:
                img[i,j,1]=int(math.ceil((img[i-1,j,1]/4.0)+(img[i,j+1,1]/4.0)+(img[i+1,j,1]/4.0)+(img[i,j-1,1]/4.0)));
    
    for i in range(2,img.shape[0],2):
        for j in range(2,img.shape[1],2):
            if i+1<img.shape[0] and j+1<img.shape[1]:
                img[i,j,1]=int(math.ceil((img[i-1,j,1]/4.0)+(img[i,j+1,1]/4.0)+(img[i+1,j,1]/4.0)+(img[i,j-1,1]/4.0)));
    
    img[0,0,1]=int(math.ceil(img[0,1,1]/2.0+img[1,0,1]/2.0))
    
    for j in range(2,img.shape[1]-1,2):
        img[0,j,1]=int(math.ceil((img[0,j-1,1]/3.0)+(img[0,j+1,1]/3.0)+(img[1,j,1]/3.0)))
    
    for i in range(2,img.shape[0]-1,2):
        img[i,0,1]=int(math.ceil((img[i-1,0,1]/3.0)+(img[i+1,0,1]/3.0)+(img[i,1,1]/3.0)))
    
    if img.shape[0]%2==0:
        img[img.shape[0]-1,img.shape[1]-1,1]=int(math.ceil((img[img.shape[0]-2,img.shape[1]-1,1]/2.0)+(img[img.shape[0]-1,img.shape[1]-2,1]/2.0)));
        for j in range(1,img.shape[1]-1,2):
            img[img.shape[0]-1,j,1]=int(math.ceil((img[img.shape[0]-1,j-1,1]/3.0)+(img[img.shape[0]-1,j+1,1]/3.0)+(img[img.shape[0]-2,j,1]/3.0)))
    else:
        img[img.shape[0]-1,0,1]=int(math.ceil((img[img.shape[0]-2,0,1]/2.0)+(img[img.shape[0]-1,1,1]/2.0)));
        for j in range(2,img.shape[1]-1,2):
            img[img.shape[0]-1,j,1]=int(math.ceil((img[img.shape[0]-1,j-1,1]/3.0)+(img[img.shape[0]-1,j+1,1]/3.0)+(img[img.shape[0]-1,j,1]/3.0)))
    
    if img.shape[1]%2==0:
        for i in range(1,img.shape[0]-1,2):
            img[i,img.shape[1]-1,1]=int(math.ceil((img[i-1,img.shape[1]-1,1]/3.0)+(img[i+1,img.shape[1]-1,1]/3.0)+(img[i,img.shape[1]-2,1]/3.0)))
    else:
        for j in range(2,img.shape[1]-1,2):
            img[i,img.shape[1]-1,1]=int(math.ceil((img[i-1,img.shape[1]-1,1]/3.0)+(img[i+1,img.shape[1]-1,1]/3.0)+(img[i,img.shape[1]-2,1]/3.0)))
#For Blue layer
    
    for i in range(1,img.shape[0],2):
        for j in range(2,img.shape[1],2):
            if j+1<img.shape[1]:
                img[i,j,2]=int(math.ceil((img[i,j-1,2]/2.0)+(img[i,j+1,2]/2.0)));
    
    for i in range(2,img.shape[0],2):
        for j in range(1,img.shape[1],2):
            if i+1<img.shape[0]:
                img[i,j,2]=int(math.ceil((img[i-1,j,2]/2.0)+(img[i+1,j,2]/2.0)));
    
    for i in range(2,img.shape[0],2):
        for j in range(2,img.shape[1],2):
            if i+1<img.shape[0] and j+1<img.shape[1]:
                img[i,j,2]=int(math.ceil((img[i-1,j-1,2]/4.0)+(img[i-1,j+1,2]/4.0)+(img[i+1,j-1,2]/4.0)+(img[i+1,j+1,2]/4.0)));
    
    img[0,:,2]=img[1,:,2]
    img[:,0,2]=img[:,1,2]
    img[0,0,2]=img[1,1,2]
    
    return img;

**Generating Demosaiced image, and storing them in "Predicted" folder**

In [ ]:
Images=[]
for i in images: 
    Images.append(Demosaicing(i))
for i in range(0,len(Images),1):
    cv2.imwrite("Predicted/"+str(i+1)+".png",cv2.cvtColor(Images[i], cv2.COLOR_BGR2RGB));

**Defining function for Mean square error and PSNR computation**

In [ ]:
def mse(o,p):
    error=0.0;
    for k in range(0,o.shape[2],1):
        for i in range(0,o.shape[0],1):
            for j in range(0,o.shape[1],1):
                e=o[i,j,k]-p[i,j,k];
                error+=e*e;
    return error/(o.shape[0]*o.shape[1]*o.shape[2])

def psnr(mse):
	return 10*math.log10((255*255)/mse)

**Showing some Demosaiced Images**

In [ ]:
f, ax = plt.subplots(2,2,figsize=(18,18))
ax[0,0].imshow(images[0])
ax[0,0].set_title("Image 1")
ax[0,1].imshow(images[0][:][:][0])
ax[0,1].set_title("Image 2")
ax[1,0].imshow(images[0][:][:][1])
ax[1,0].set_title("Image 3")
ax[1,1].imshow(images[0][:][:][2])
ax[1,1].set_title("Image 4")

**Computing PSNR for Demosaiced images with respect to original rgb images.**

In [ ]:
for i in range(0,50,1):
    print("PSNR for image "+str(i+1)+".png "+str(psnr(mse(oimages[i],Images[i]))));

**Defining Bilateral filter**

In [ ]:
def bilateral_filter(X_matrix,size,sigma_d,sigma_r):
	X=np.zeros((len(X_matrix),len(X_matrix[0])))
	center=size//2    
	for j in range(center,len(X_matrix)-center,1):
		for k in range(center,len(X_matrix[0])-center,1):
			temp=0;
			w_sum=0;
			for m in range(-center,center+1,1):
				for n in range(-center,center+1,1):
					w=math.exp(-(((m)**2+(n)**2)/(2*sigma_d*sigma_d))-((abs(X_matrix[j+m][k+n]-X_matrix[j][k]))/(2*sigma_r*sigma_r)))
					temp+=X_matrix[j+m][k+n]*w;
					w_sum+=w;
			X[j][k]=temp//(w_sum);
	return X;

**Applying Bilateral filter on Demosaiced images**

In [ ]:
bImages=[]
for i in Images:
    i[:,:,0]=bilateral_filter(i[:,:,0],3,2,0.25)
    i[:,:,1]=bilateral_filter(i[:,:,1],3,2,0.25)
    i[:,:,2]=bilateral_filter(i[:,:,2],3,2,0.25)
    bImages.append(i);

**Storing Images are after applying Bilateral filter.**

In [ ]:
for i in range(0,len(bImages),1):
    cv2.imwrite("Bilateral/"+str(i+1)+".png",cv2.cvtColor(bImages[i], cv2.COLOR_BGR2RGB));

**Computing PSNR for demosaiced images after applying Bilateral filter.**

In [ ]:
for i in range(0,50,1):
    print("PSNR for image "+str(i+1)+".png "+str(psnr(mse(oimages[i],bImages[i]))));

**After applying Bilateral filter,PSNR of Images reduced.**